In [2]:
!pip install selenium
!pip install webdriver-manager

In [1]:
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from tqdm import tqdm

In [2]:
addr = "https://www.boxofficemojo.com/search/?q=Think+Like+a+Dog" 

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(addr)

stories = []
a_release = []
o_release = []

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [C:\Users\beomj\.wdm\drivers\chromedriver\win32\84.0.4147.30\chromedriver.exe] found in cache
 


In [3]:
df2 = pd.read_excel("1970_2019_영화.xlsx")
df2.prdtYear.fillna(0, inplace=True)
years = df2.prdtYear.astype(int).astype(str)
names = df2.movieNmEn	

In [8]:
df = pd.DataFrame({"영화명EN": names})
df['스토리']=None
df['A_release']=None
df['O_release']=None
df

,영화명EN,스토리,A_release,O_release
0,Think Like a Dog,None,None,None
1,The New Mutants,None,None,None
2,Drive,None,None,None
3,Jurassic Thunder,None,None,None
4,Sea Monsters2,None,None,None
...,...,...,...,...
5820,Hanna'S War,None,None,None
5821,Hearts Of Fire,None,None,None
5822,Hot Target,None,None,None
5823,Hamberger Hill,None,None,None


In [5]:
def find(name, year):
    stories = []
    a_release = []
    o_release = []
    try:
        driver.find_element_by_id("mojo-search-text-input").send_keys(name)
        driver.find_element_by_id("mojo-search-button").click()
        while True:
            try:
                time.sleep(1)
                driver.find_element_by_id("body")
                break
            except:
                pass
        temp = driver.find_element_by_xpath("/html/body/div[1]/main/div/div/div")
        temp = temp.find_elements_by_class_name("a-fixed-left-grid-inner")
        found = False
        for i in temp:
            if year in i.text:
                i.find_element_by_tag_name("a").click()
                found =True
                break
        if not found:
            for i in temp:
                if str(int(year)-1) in i.text or str(int(year)+1) in i.text:
                    i.find_element_by_tag_name("a").click()
                    found =True
                    break
        if not found:
            stories.append("")
            a_release.append("")
            o_release.append("")
            return stories, a_release, o_release
        #-----------------------
        try:
            stories.append(driver.find_element_by_class_name("mojo-heading-summary").text)
        except:
            stories.append("")
        #-----------------------
        try:
            a_release.append(driver.find_element_by_class_name("mojo-summary-table").text)
        except:
            a_release.append("")
        #------------------------
        try:
            temp = driver.find_element_by_class_name("a-dropdown-prompt").click()
            driver.find_element_by_link_text("Original Release").click()
            while True:
                try:
                    time.sleep(0.7)
                    total = []
                    table = driver.find_elements_by_class_name("mojo-table")
                    for i in table:
                        total.append(i.text)
                    o_release.append("\n".join(total))
                    if table:
                        break
                except:
                    pass
        except:
            o_release.append("")
        #-----------------------
    except:
        stories.append("")
        a_release.append("")
        o_release.append("")
    driver.get(addr)
    return stories, a_release, o_release

In [9]:
start = 4800
end = len(df)
for i in tqdm(range(start, end)):
    name = names[i]
    year = years[i]
    s, a, o = find(name, year) 
    df.loc[i, '스토리'] = s[0]
    df.loc[i, 'A_release'] = a[0]
    df.loc[i, 'O_release'] = o[0]        
    

100%|██████████| 1025/1025 [1:39:29<00:00,  5.82s/it]


In [10]:
df.to_excel("1970_2019_mojo"+str(start)+"_"+str(end)+".xlsx", index=False)

In [12]:
df[4800:].A_release.isna().sum()

0

In [16]:
(df[4800:].A_release=="").sum()

258

In [ ]:
(df[4800:].A_release=="")